# Importar librerias

In [6]:
import cv2
import wandb
from ultralytics import YOLO

# Cargamos el modelo

In [7]:
model = YOLO('runs/detect/train7/weights/best.pt')

# Función para predecir sobre video

In [10]:
def predict_and_export_video(model, input_video_path, output_video_path):
    # Cargar el video de entrada
    cap = cv2.VideoCapture(input_video_path)
    
    # Obtener propiedades del video
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Configurar el escritor de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para el formato .mp4
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Realizar la predicción
        results = model(frame)
        
        # Dibujar las predicciones en el frame
        for result in results:
            boxes = result.boxes  # Lista de bounding boxes detectadas
            for box in boxes:
                # Obtener las coordenadas de la caja y la clase detectada
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordenadas de la caja
                class_id = int(box.cls[0])  # ID de la clase predicha
                score = box.conf[0]  # Confianza de la predicción
                
                # Dibujar la caja en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label = f"{model.names[class_id]}: {score:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Escribir el frame procesado en el video de salida
        out.write(frame)
    
    # Liberar los recursos
    cap.release()
    out.release()
    print(f"Video exportado exitosamente como {output_video_path}")

In [ ]:
predict_and_export_video(model, 'senales2.mp4', 'output.mp4')

# Función par detectar en tiempo real

In [ ]:
def real_time_prediction(model):
    # Iniciar la captura de video desde la cámara web (0 indica la cámara por defecto)
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se pudo capturar el frame de la cámara.")
            break
        
        # Realizar la predicción en el frame capturado
        results = model(frame)
        
        # Dibujar las predicciones en el frame
        for result in results:
            boxes = result.boxes  # Lista de bounding boxes detectadas
            for box in boxes:
                # Obtener las coordenadas de la caja y la clase detectada
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordenadas de la caja
                class_id = int(box.cls[0])  # ID de la clase predicha
                score = box.conf[0]  # Confianza de la predicción
                
                # Dibujar la caja en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label = f"{model.names[class_id]}: {score:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Mostrar el frame en una ventana de tiempo real
        cv2.imshow('YOLO Real-Time Detection', frame)
        
        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()